Estudiantes:

    Jason Solano
    
    Percy Herrera

# Descripción del Dataset:

Para el dataset se evalue el siguiente caso en la plataforma Kaggle:

https://www.kaggle.com/corrieaar/apartment-rental-offers-in-germany/data#

El dataset consite en poder visualizar el precio total de la renta de una region determinada en Alemania por medio de las siguientes variables, como se puede observar es un problema de regression.


Consiste en las siguientes variables:

**regio1:** 
Bundesland

**serviceChargeaucilliary:** 
costs such as electricty or internet in €

**heatingType:** 
Type of heating

**telekomTvOfferIs:**
payed TV included if so which offer

**telekomHybridUploadSpeedhow:**  
fast is the hybrid inter upload speed

**newlyConstis:**
the building newly constructed

**balconydoes:**
the object have a balcony

**electricityBasePricemonthly:**
base price for electricity in €

**picturecounthow:**
many pictures were uploaded to the listing

**pricetrendpriceL:**
trend as calculated by Immoscout

**telekomUploadSpeedhow:**
fast is the internet upload speed

**totalRenttotal:**
rent (usually a sum of base rent, service charge and heating cost)

**yearConstructedconstruction:**
year

**electricityKwhPriceelectricity:**
price per kwh

**scoutIdimmoscout:**
Id

**noParkSpaces:**
number of parking spaces

**firingTypes:** 
main energy sources, separated by colon

**hasKitchen:** 
has a kitchen

**geo_bln:** 
bundesland, same as regio1

**cellar:**
has a cellar

**yearConstructedRange:**
binned construction year, 1 to 9

**baseRentbase:**
rent without electricity and heating

**houseNumber**
house number

**livingSpace:**
living space in sqm

**geo_krs:** 
district, above ZIP code

**condition:** 
condition of the flat

**interiorQual** 
interior quality

**petsAllowed** 
are pets allowed, can be yes, no or negotiable

**streetPlain** 
street name

**lift** is elevator available

**baseRentRange:** 
binned base rent, 1 to 9

**typeOfFlat:**
type of flat

**geo_plz:**
ZIP code

**noRooms:**
number of rooms

**thermalChar:**
energy need in kWh/(m^2a), defines the energy efficiency class

**floor:** which floor is the flat on

**numberOfFloor:** snumber of floors in the building

**noRoomsRange** 
binned number of rooms, 1 to 5

**garden**
has a garden

**livingSpaceRange** 
binned living space, 1 to 7

**regio2** 
District or Kreis, same as geo krs

**regio3** 
City/town

**description**
free text description of the object

**facilities** 
free text description about available facilities

**heatingCosts** 
monthly heating costs in €

**energyEfficiencyClass:**
class (based on binned thermalChar)

**lastRefurbish:**
year of last renovation

**date:**
time of scraping

# Procesamiento de Datos

### Debido a un problema de la libreria de pyspark confunde datos de las columnas: 'Description' y 'facilities' ambas columnas contienen datos de descripciones y beneficios de la vivienda para la renta, por lo cual no es muy beneficioso para los algoritmos que utilizaremos, a no ser que se intente utilizar algoritmos NLP, los cuales salen del alcance de dicho ejercicio.


### Se utilizara Pandas para eliminar dichas columnas

In [10]:
import pandas as pd
file = pd.read_csv('immo_data.csv')
del file['description']
del file['facilities']
file.dtypes
file.to_csv('immo_dataML.csv',sep=';')

### Utilizamos pyspark para tratar las otras variables

Importamos la librerias requeridas y la configuración para conectar con postgres

In [12]:
import findspark
findspark.init('/opt/spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

#path del archivo postgresql-42.1.4.jar
dir = "/Users/jasonsolano/Documents/BigData/BigDataTEC/Clase1/DB/"
spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", dir+"postgresql-42.1.4.jar") \
    .config("spark.executor.extraClassPath", dir+"postgresql-42.1.4.jar") \
    .getOrCreate()

Leamos la tabla  apartmentrental que contiene los atributos necesarios para guardar los archivos del CVS

In [13]:
dfT = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/ImmoDataBase") \
    .option("user", "postgres") \
    .option("password", "102800") \
    .option("dbtable", "apartmentrental") \
    .load()

dfT.show()

+-------+--------+-------------+------+---------+--------------------+-------------------+---------------------+-----------+-----+------+-------+-------+-------+----------+-----------+------------+-----------+---+------------+-------------+----+-----------+----------------+----------+------------+-------+------------+--------------+-----------+------------+----------+------+------+------+-------+-------------+-----------+------------------------+--------------+------------------+-----------+---------+----------+---------------+--------------------+----+
|balcony|baserent|baserentrange|cellar|condition|electricitybaseprice|electricitykwhprice|energyefficiencyclass|firingtypes|floor|garden|geo_bln|geo_krs|geo_plz|haskitchen|heatingtype|heatingcosts|housenumber| id|interiorqual|lastrefurbish|lift|livingspace|livingspacerange|newlyconst|noparkspaces|norooms|noroomsrange|numberoffloors|petsallowed|picturecount|pricetrend|regio1|regio2|regio3|scoutid|servicecharge|streetplain|telekomhybridup

### Se lee el CSV que habia sido procesado por pandas con la libreria pyspark

In [14]:
#read the dataset
session=SparkSession.builder.appName('EDAPart1').getOrCreate()
df=session.read.options(delimiter=';').csv('immo_dataML.csv',header=True,inferSchema =True)
# dicha columna es generada por pardas, por eso se borra
columns_to_drop = ['_c0']
df = df.drop(*columns_to_drop)


In [15]:
df.printSchema()

root
 |-- regio1: string (nullable = true)
 |-- serviceCharge: double (nullable = true)
 |-- heatingType: string (nullable = true)
 |-- telekomTvOffer: string (nullable = true)
 |-- telekomHybridUploadSpeed: double (nullable = true)
 |-- newlyConst: boolean (nullable = true)
 |-- balcony: boolean (nullable = true)
 |-- electricityBasePrice: double (nullable = true)
 |-- picturecount: integer (nullable = true)
 |-- pricetrend: double (nullable = true)
 |-- telekomUploadSpeed: double (nullable = true)
 |-- totalRent: double (nullable = true)
 |-- yearConstructed: double (nullable = true)
 |-- electricityKwhPrice: double (nullable = true)
 |-- scoutId: integer (nullable = true)
 |-- noParkSpaces: double (nullable = true)
 |-- firingTypes: string (nullable = true)
 |-- hasKitchen: boolean (nullable = true)
 |-- geo_bln: string (nullable = true)
 |-- cellar: boolean (nullable = true)
 |-- yearConstructedRange: double (nullable = true)
 |-- baseRent: double (nullable = true)
 |-- houseNumber

### Escribimos los datos en la tabla de postgres

In [16]:
df.write.format('jdbc').options(
      url='jdbc:postgresql://localhost/ImmoDataBase',
      dbtable='apartmentrental',
      user='postgres',
      password='102800').mode('overwrite').save()